@author: Alessandro Lovo

# Miscellaneous plots

This notebook contains miscellaneous plots / analysis. See the headers for more detail

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib widget
matplotlib.rc('font', size=18)
default_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

import xarray as xr
import pandas as pd
from scipy import sparse
from tqdm.notebook import tqdm

import sys
sys.path.append('../Climate-Learning/')

import general_purpose.uplotlib as uplt
import general_purpose.cartopy_plots as cplt
import general_purpose.utilities as ut

HOME = './'

## Visualize the data normalization procedure

In [ ]:
lon = np.load('common/lon.npy')
lat = np.load('common/lat.npy')
LON, LAT = np.meshgrid(lon,lat)

y = 'r800y' # use the networks trained on 800 years of data

X_means = np.stack([np.load(f'common/{y}/fold_{fold}/X_mean.npy') for fold in range(5)])
X_stds = np.stack([np.load(f'common/{y}/fold_{fold}/X_std.npy') for fold in range(5)])

X_stds[X_stds == 1] = np.nan

X_means.shape, X_stds.shape

In [ ]:
mfp_kwargs = dict(one_fig_layout=120, figsize=(10,5),
                  projections=[cplt.ccrs.Orthographic(central_latitude=90), cplt.ccrs.PlateCarree()],
                  extents=[None, (-5, 10, 39, 55)],
                  titles=[r'Geopotential height [m]', 'Soil moisture [$m^3/m^3$]'],
                 )
cmaps=['RdBu_r', 'BrBG']

std_kwargs = dict(cmaps=['Reds', 'Greens'], vmin=0, colorbar='disabled')

In [ ]:
for fold in range(5):
    _ = cplt.mfp(LON,LAT,X_means[fold], **mfp_kwargs, fig_num=8+fold, cmaps=cmaps)
    fig = _[0].get_figure()
    fig.suptitle(f'fold {fold}')

Since the fields are already anomalies, the means are very close to 0

In [ ]:
for fold in range(5):
    _ = cplt.mfp(LON,LAT,X_stds[fold], **mfp_kwargs, fig_num=8+fold, **std_kwargs)
    fig = _[0].get_figure()
    fig.suptitle(f'fold {fold}')

In [ ]:
_ = cplt.mfp(LON,LAT,X_stds.mean(axis=0), **mfp_kwargs, fig_num=1, **std_kwargs)
fig = _[0].get_figure()
fig.suptitle(f'Mean of stds')
_ = cplt.mfp(LON,LAT,X_stds.std(axis=0), **mfp_kwargs, fig_num=2, cmaps=['Reds', 'Greens'], vmin=0, colorbar='disabled')
fig = _[0].get_figure()
fig.suptitle(f'Std of stds')

The maps of std across the folds are extremely similar, we can show only one

In [ ]:
fold = 0
_ = cplt.mfp(LON,LAT,X_stds[fold], **mfp_kwargs, fig_num=8+fold, **std_kwargs)
fig = _[0].get_figure()
fig.suptitle(f'pixel-wise standard deviation')

# fig.tight_layout(w_pad=0)
fig.tight_layout()

fig.savefig(f'{HOME}/STD.pdf')
# fig.savefig(f'{HOME}/STD.png', dpi=300)

## Quantify how many independent heatwave samples we have

Since we take $A(t)$ as the $T=14$-day long running mean of the 2m temperature anomaly over France, consecutive samples are necessarily correlated

In [ ]:
A_te = xr.open_dataarray('common/A_te.nc')
threshold = np.load('common/threshold.npy')
A_te

In [12]:
A = A_te.data.reshape(200,-1) # heatwave amplitude for each year
Y = (A >= threshold) # heatwave labels for each year

We define a `heatwave day` $d(t)$ which increases by one for each consecutive day that $A(t) \geq a$, i.e. when we are inside a heatwave.
When we exit the heatwave ($A(t) < a$), $d(t)$ goes back to 0, and we save in `heatwave duration` the total number of consecutive days $D(t)$ for which the 14-day average temperature anomaly was above the threshold.

We call this variable `heatwave duration`, but beware of the interpretation: if the 14-day average temperature anomaly was above $a$ for $D$ consecutive day, then we can say that the $D+14$-day average temperature was also likely to be above $a$, so, striclty speaking, the heatwave lasted around $D+14$ days.

In [ ]:
heatwave_day = np.zeros_like(A)
heatwave_durations = []
for y in tqdm(range(Y.shape[0])): # iterate over the years
    for i in range(Y.shape[1]):
        if i == 0 or Y[y,i] == 0:
            heatwave_day[y,i] = Y[y,i]
            if i > 0 and heatwave_day[y,i-1] > 0:
                heatwave_durations.append(heatwave_day[y,i-1])
        else:
            heatwave_day[y,i] = heatwave_day[y,i-1] + 1

        if i == (Y.shape[1] - 1) and heatwave_day[y,i] > 0:
            heatwave_durations.append(heatwave_day[y,i])

assert np.sum(Y) == np.sum(heatwave_day > 0) == np.sum(heatwave_durations)

In [ ]:
y = 0

plt.close(1)
fig, ax = plt.subplots(num=1, figsize=(9,6))

plt.plot(A[y], color='black', label='$A$')
plt.plot(Y[y], label='$Y$')
plt.plot(heatwave_day[y], label='heatwave day')
plt.axhline(threshold, color='red')

plt.legend()
plt.xlabel('time [days]')
plt.title(f'Test year {y}')

fig.tight_layout()

In [ ]:
plt.close(2)
fig, ax = plt.subplots(num=2, figsize=(9,6))

hist, bins, _ = plt.hist(heatwave_day.flatten(),
                         bins=np.arange(1,Y.shape[1]+1), # excluding 0 so we ignore all non-heatwave data
                         label='heatwave day')
plt.hist(heatwave_durations, bins=bins, label='heatwave duration')

plt.yscale('log')

plt.legend()

fig.tight_layout()

In [ ]:
hist[0]/np.sum(hist)

Days of heatwave (i.e. days for which $A(t) > a$) that are also the first day of a longer heatwave (and thus can be considered independent) constitute only 13.8% of the total number of days of heatwave.
So yes, in our prediction task we are mainly focused on predicting the continuation of an already ongoing heatwave.